In [1]:
#here really understand the dp dmu (mu prime , imax)
import matplotlib.pyplot as plt
import numpy as np
import scipy as scp
from scipy.integrate import quad
from multiprocessing import Pool

import cst
from math import pi, log10, sqrt, atan, tan, sin, cos, asin, acos, erf, floor

Ncpu = 8  # manually set the number of logical cores


def cot(x):
    return 1./tan(x)

In [13]:
#CDF Time
def cdf():
    i_max = 30
    mup  = np.linspace(0,.5,5)
    mu = np.linspace(0,1,5)
    maxa =  1
    mini = np.cos(i_max)
    y = 0 
    x = min((mup/np.sqrt(1-mu*mu)),1)
    x= np.arcsin(x)
    res = quad(x,mini,maxa)
    plt.plot(res)
        

In [35]:
i_max = 30
mup  = np.linspace(0,.5,100)
maxa =  1
mini = np.cos(i_max)
y = 0 
res_tot = [] 

for mup_part in mup:
        def integrand(t):
              return np.arcsin(min((mup_part/np.sqrt(1-t*t)),1))
        res, err = quad(integrand,mini,maxa)
        res_tot.append(res)
# shes a lil broken

In [2]:
# this is the real meat <3 
#we are sampling in a primed coordinate system and going to the primed

#params
case_paras = [2, 50, 50] # M star, in solar mass, m bh total in m sun, vk mean km/s
m = case_paras[0]
M = case_paras[1]

#distribution of phi, psi, and kick velocity

#definiotns
def R_star(M):
    if M > 1:
        return 0.0046524726 * M**0.6
    else:
        return 0.0046524726 * M**0.8
    
rt = R_star(M) *(m/M)**(1/3) # tidal radii depedence for mergers
Nlga, Nlgrp = 10, 10  # low (30, 30); medium (100, 100) ; high resolution (200, 200)


In [3]:
# here goes all the for loops for the probabilities
# distribution for the kick amplitudes, all in log space to make computation easier
# this is the cdf of the vks, the vk params, and the differences of the vks used for later calculations

vk_mean = np.log10(case_paras[2]*1e5/2*pi*1.496e8/3.154e7)
vk_sigma = .3 # vary this param as needed to determine the spread of the velocities 
vk_min = vk_mean - 5*vk_sigma
vk_max = vk_mean + 5*vk_sigma 
vks = np.linspace(vk_min,vk_max,1000) # vary number of steps as needed for computational speeds

cdf_vks_arr = np.array([.5 +.5*erf((vk-vk_mean)/np.sqrt(2)*vk_sigma) for vk in vks])
cdf_vks_diff = np.diff(cdf_vks_arr)

amin, amax = .5,100 # look at why this is
rpmin, rpmax = .3,amax # below .1 au may be unstable

merg_lgra_arr = np.empty(Nlga,dtype = float)

#set up grid :0
# the phi grid concentrates near phi=0 and phi=pi
Nphi_half = 30  # 30 should be sufficient
dphi_half = pi/(Nphi_half*(Nphi_half+1))*(np.arange(Nphi_half)+1)
dphi_arr_p = np.concatenate((dphi_half, np.flip(dphi_half)))
phi_arr_p = np.cumsum(dphi_arr_p) - dphi_arr_p/2  # put at the middle of each bin
dParr_phi_p = dphi_arr_p/pi    # probability for each bin dP/dphi*dphi
Nphi_p = 2*Nphi_half

# the psi grid for orbital phase  (with slight concentration near psi=pi)
Npsi = 50  # 50 should be sufficient
dpsi_min = 2*pi/(5*Npsi)    # dpsi_min = dpsi_max/4
dpsi_arr = dpsi_min + 3*dpsi_min/(Npsi-1) * np.arange(Npsi)
psi_arr = np.cumsum(dpsi_arr) - dpsi_arr/2  # put at the middle of each bin
dParr_psi = np.empty(Npsi, dtype=float)

# kick velocity amplitude distribution (log-normal)
vunit2cgs = 2*pi*1.496e8/3.154e7
lgvk_mean = log10(case_paras[2]*1e5/vunit2cgs)  # amplitude of the kick in machine units
lgvk_sigma = 0.3
Nlgvk = 1000    # this high resolution does not add to computational burden
lgvk_min, lgvk_max = lgvk_mean - 5*lgvk_sigma, lgvk_mean + 5*lgvk_sigma
lgvk_arr = np.linspace(lgvk_min, lgvk_max, Nlgvk, endpoint=True)
dlgvk = lgvk_arr[1] - lgvk_arr[0]
CDF_lgvk_arr = np.array([0.5 + 0.5*erf((lgvk-lgvk_mean)/(sqrt(2)*lgvk_sigma))
                         for lgvk in lgvk_arr])
CDF_lgvk_diff = np.diff(CDF_lgvk_arr)
lgamin, lgamax = log10(amin), log10(amax)
lga_arr = np.linspace(lgamin, lgamax, Nlga, endpoint=True)
lgrpmin, lgrpmax = log10(rpmin), log10(rpmax)
lgrp_arr = np.linspace(lgrpmin, lgrpmax, Nlgrp, endpoint=True)
eps_small = 1e-10   # very small floor number
lgf_floor = log10(eps_small)
inc = np.linspace(0,30,1000)


In [4]:
# here goes all the for loops for the probabilities
# distribution for the kick amplitudes, all in log space to make computation easier
# this is the cdf of the vks, the vk params, and the differences of the vks used for later calculations

vk_mean = np.log10(case_paras[2]*1e5/2*pi*1.496e8/3.154e7)
vk_sigma = .3 # vary this param as needed to determine the spread of the velocities 
vk_min = vk_mean - 5*vk_sigma
vk_max = vk_mean + 5*vk_sigma 
vks = np.linspace(vk_min,vk_max,1000) # vary number of steps as needed for computational speeds

cdf_vks_arr = np.array([.5 +.5*erf((vk-vk_mean)/np.sqrt(2)*vk_sigma) for vk in vks])
cdf_vks_diff = np.diff(cdf_vks_arr)

amin, amax = .5,100 # look at why this is
rpmin, rpmax = .3,amax # below .1 au may be unstable

merg_lgra_arr = np.empty(Nlga,dtype = float)

#set up grid :0
# the phi grid concentrates near phi=0 and phi=pi
Nphi_half = 30  # 30 should be sufficient
dphi_half = pi/(Nphi_half*(Nphi_half+1))*(np.arange(Nphi_half)+1)
dphi_arr_p = np.concatenate((dphi_half, np.flip(dphi_half)))
phi_arr_p = np.cumsum(dphi_arr_p) - dphi_arr_p/2  # put at the middle of each bin
dParr_phi_p = dphi_arr_p/pi    # probability for each bin dP/dphi*dphi
Nphi_p = 2*Nphi_half

# the psi grid for orbital phase  (with slight concentration near psi=pi)
Npsi = 50  # 50 should be sufficient
dpsi_min = 2*pi/(5*Npsi)    # dpsi_min = dpsi_max/4
dpsi_arr = dpsi_min + 3*dpsi_min/(Npsi-1) * np.arange(Npsi)
psi_arr = np.cumsum(dpsi_arr) - dpsi_arr/2  # put at the middle of each bin
dParr_psi = np.empty(Npsi, dtype=float)

# kick velocity amplitude distribution (log-normal)
vunit2cgs = 2*pi*1.496e8/3.154e7
lgvk_mean = log10(case_paras[2]*1e5/vunit2cgs)  # amplitude of the kick in machine units
lgvk_sigma = 0.3
Nlgvk = 1000    # this high resolution does not add to computational burden
lgvk_min, lgvk_max = lgvk_mean - 5*lgvk_sigma, lgvk_mean + 5*lgvk_sigma
lgvk_arr = np.linspace(lgvk_min, lgvk_max, Nlgvk, endpoint=True)
dlgvk = lgvk_arr[1] - lgvk_arr[0]
CDF_lgvk_arr = np.array([0.5 + 0.5*erf((lgvk-lgvk_mean)/(sqrt(2)*lgvk_sigma))
                         for lgvk in lgvk_arr])
CDF_lgvk_diff = np.diff(CDF_lgvk_arr)
lgamin, lgamax = log10(amin), log10(amax)
lga_arr = np.linspace(lgamin, lgamax, Nlga, endpoint=True)
lgrpmin, lgrpmax = log10(rpmin), log10(rpmax)
lgrp_arr = np.linspace(lgrpmin, lgrpmax, Nlgrp, endpoint=True)
eps_small = 1e-10   # very small floor number
lgf_floor = log10(eps_small)

#uniform distribution of inclination angles from 0,30 or whatever u feel like :)
inc = np.linspace(0,np.pi/6,1000) # or whaterves in radians !!!
lgvk_mean = log10(case_paras[2]*1e5/vunit2cgs)  # amplitude of the kick in machine units
lgvk_sigma = 0.3
Nlgvk = 1000    # this high resolution does not add to computational burden
lgvk_min, lgvk_max = lgvk_mean - 5*lgvk_sigma, lgvk_mean + 5*lgvk_sigma
lgvk_arr = np.linspace(lgvk_min, lgvk_max, Nlgvk, endpoint=True)
dlgvk = lgvk_arr[1] - lgvk_arr[0]
CDF_lgvk_arr = np.array([0.5 + 0.5*erf((lgvk-lgvk_mean)/(sqrt(2)*lgvk_sigma))
                         for lgvk in lgvk_arr])
CDF_lgvk_diff = np.diff(CDF_lgvk_arr)

def calc_lgfmerg_for_each_lgrp(lgrp):
    rp = 10**lgrp
    lgfmerg_lgra_arr = np.empty(Nlga, dtype=float)
    for i_a  in range(Nlga):
        a = 10 **lga_arr[i_a] 
        if a < rp: # unphysicsal
            lgfmerg_lgra_arr[i_a] = lgf_floor
            continue
        e = max(1-rp/a,1e-5)
        for eee in range(inc):
            for i in range(Npsi): #psi is the orbital phase
                dP_psi = dParr_psi[i] 
                psi = psi_arr[i] + dpsi_arr[i]/2    # right boundary of each bin
                if i == Npsi - 1:   # the precise right boundary psi = pi (where tan(psi/2) diverges)
                    temp_CDF_right = 1.
                else:
                    temp_CDF_right = 1/pi * (2*atan(sqrt((1-e)/(1+e)) * tan(psi/2)) - e*sqrt(1-e*e)*sin(psi)/(1+e*cos(psi)))
                dParr_psi[i] = temp_CDF_right - temp_CDF_left
                temp_CDF_left = temp_CDF_right
            # from merger time = Hubble time
                fc = (2*rt)/(a*(1-e*e))
                temp_phi = 0

            # then calculate merger fraction for each set of (a, e)
                fmerg = 1e-10
                r = (a * (1- e*e))/(1+e*np.cos(psi))*np.tan(eee)
                v = sqrt((M+m)*(2./r - 1./a)) 
                vx = v * np.sin(i)
                vy = v * np.cos(i)
                alpha = asin((1+e*cos(psi))/sqrt(e*e + 2*e*cos(psi) + 1))  # (0, pi/2)

                # now things get weird ! all of this is in primed coordiantes and need to convert
                for j in range(Nphi_p):
                    phi_p = phi_arr_p[j]
                    phi = np.arccos(2*(1/np.tan(phi_p)-1/(np.sin(phi_p)*np.cos(phi_p)))*np.tan(np.arccos(vx * np.sin(the_p)* np.cos(phi_p)+vy * np.sin(the_p)*np.sin(phi_p))))
                    dP_phi = dParr_phi_p[j]   # probability for this dphi bin
                
                    xi = cot(alpha)*cos(phi) - sqrt(1./fc - 1)*sin(phi)/sin(alpha)
                    mu_max = xi/sqrt(1+xi*xi)
                    dmu = (mu_max+1)/50 
                    mu_p = -1 + dmu/2
                    the_p = np.arccos(mu_p)
                    dP_mu = np.arcsin(min((mu_p/(np.sqrt(1-mup*mup))),1))  # probability for each dmu bin
                    temp_mu = 0.
                    for k in range(mu_p):
                        the_p = np.arccos(mu_p)
                        mu =  vx* np.sin(the_p)*np.cos(phi_p) + vy * np.sin(the_p)*np.sin(phi_p)
                        the = np.arccos(mu)
                        ymax = sqrt(mu*mu + r/(2*a-r)) - mu     # avoid unbound orbit
                        eta = (sin(phi)/sin(alpha)/(cot(the) - cot(alpha)*cos(phi)))**2
                        s2 = cos(the) - cot(alpha)*sin(the)*cos(phi)
                        yplus = min((1 + sqrt((eta + 1)*fc - eta))/(-s2*(eta+1)), ymax)
                        yminus = (1 - sqrt((eta + 1)*fc - eta))/(-s2*(eta+1))
                        lgvk_plus = min(log10(yplus*v), lgvk_max)
                        lgvk_minus = max(log10(yminus*v), lgvk_min)
                        if lgvk_plus < lgvk_min or lgvk_minus > lgvk_max or lgvk_plus < lgvk_minus:
                            mu_p += dmu
                            continue    # no such y for merger to happen
                        i_floor = int(floor((lgvk_plus-lgvk_min)/dlgvk))
                        CDF_lgvk_plus = CDF_lgvk_arr[i_floor] \
                            + CDF_lgvk_diff[i_floor]/dlgvk*(lgvk_plus-lgvk_arr[i_floor])
                        i_floor = int(floor((lgvk_minus-lgvk_min)/dlgvk))
                        CDF_lgvk_minus = CDF_lgvk_arr[i_floor] \
                            + CDF_lgvk_diff[i_floor]/dlgvk*(lgvk_minus-lgvk_arr[i_floor])
                        temp_mu += dP_mu * (CDF_lgvk_plus - CDF_lgvk_minus)
                        mu_p += dmu
                    temp_phi += temp_mu*dP_phi
            fmerg += temp_phi*dP_psi
        lgfmerg_lgra_arr[i_a] = log10(fmerg)
    return lgfmerg_lgra_arr
                
            
                


savename = 'fmerger2D_M%d_m%.1f_vkmean%d_nov26' % (case_paras[0], case_paras[1], case_paras[2])

data_dir = '/Users/sophiarisin/Dropbox/Mac (2)/Desktop/research/wenbin/tdes'
# parallel calculation starting from here
if __name__ == '__main__':
    with Pool(Ncpu) as po:
        lgfmerg = np.array(po.map(calc_lgfmerg_for_each_lgrp, lgrp_arr))



    # write the solution into a file
    print('writing:' + data_dir+savename+'.txt')
    with open(data_dir+savename+'.txt', 'w') as f:
        f.write('lgamin\tlgamax\tNlga\t%.5f\t%.5f\t%d\n' % (lgamin, lgamax, Nlga))
        f.write('lgrpmin\tlgrpmax\tNlgrp\t%.5f\t%.5f\t%d\n' % (lgrpmin, lgrpmax, Nlgrp))
        f.write('lgfmerg(Nlga, Ne)')
        for i_a in range(Nlga):
            f.write('\n')
            for i_rp in range(Nlgrp):
                if i_rp == 0:
                    f.write('%.5f' % lgfmerg[i_rp, i_a])
                else:
                    f.write('\t%.5f' % lgfmerg[i_rp, i_a])

Process SpawnPoolWorker-1:
Process SpawnPoolWorker-2:
Process SpawnPoolWorker-3:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Process SpawnPoolWorker-4:
  File "/opt/homebrew/Cellar/python@3.11/3.11.3/Frameworks/Python.framework/Versions/3.11/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/homebrew/Cellar/python@3.11/3.11.3/Frameworks/Python.framework/Versions/3.11/lib/python3.11/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/homebrew/Cellar/python@3.11/3.11.3/Frameworks/Python.framework/Versions/3.11/lib/python3.11/multiprocessing/pool.py", line 114, in worker
    task = get()
           ^^^^^
  File "/opt/homebrew/Cellar/python@3.11/3.11.3/Frameworks/Python.framework/Versions/3.11/lib/python3.11/multiprocessing/queues.py", line 367, in get
    return _ForkingPickler.loads(res)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
Attrib

KeyboardInterrupt: 

In [24]:
savename = 'fmerger2D_M%d_m%.1f_vkmean%d_nov26' % (case_paras[0], case_paras[1], case_paras[2])

data_dir = '/Users/sophiarisin/Dropbox/Mac (2)/Desktop/research/wenbin/tdes'
# parallel calculation starting from here
if __name__ == '__main__':
    with Pool(Ncpu) as po:
        lgfmerg = np.array(po.map(calc_lgfmerg_for_each_lgrp, lgrp_arr))

    # write the solution into a file
    print('writing:' + data_dir+savename+'.txt')
    with open(data_dir+savename+'.txt', 'w') as f:
        f.write('lgamin\tlgamax\tNlga\t%.5f\t%.5f\t%d\n' % (lgamin, lgamax, Nlga))
        f.write('lgrpmin\tlgrpmax\tNlgrp\t%.5f\t%.5f\t%d\n' % (lgrpmin, lgrpmax, Nlgrp))
        f.write('lgfmerg(Nlga, Ne)')
        for i_a in range(Nlga):
            f.write('\n')
            for i_rp in range(Nlgrp):
                if i_rp == 0:
                    f.write('%.5f' % lgfmerg[i_rp, i_a])
                else:
                    f.write('\t%.5f' % lgfmerg[i_rp, i_a])

Process SpawnPoolWorker-58:
Process SpawnPoolWorker-61:
Process SpawnPoolWorker-62:
Process SpawnPoolWorker-59:
Process SpawnPoolWorker-55:
Process SpawnPoolWorker-56:
Process SpawnPoolWorker-60:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Process SpawnPoolWorker-57:
Traceback (most recent call last):
  File "/opt/homebrew/Cellar/python@3.11/3.11.3/Frameworks/Python.framework/Versions/3.11/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/homebrew/Cellar/python@3.11/3.11.3/Frameworks/Python.framework/Versions/3.11/lib/python3.11/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/homebrew/Cellar/python@3.11/3.11.3/Frameworks/Python.framework/Versions/3.11/lib/python3.11/multiprocessing/pool.

KeyboardInterrupt: 